In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [3]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [4]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [5]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [6]:
def get_annotations(song):
    referenced = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/referents?song_id='+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for ref in refs_json['response']['referents']:
        for note in ref['annotations']:
            for child in note['body']['dom']['children']:
                if 'children' in child:
                    for element in child['children']:
                        if 'attributes' in element:
                            if 'href' in element['attributes']:
                                parsed = urlparse(element['attributes']['href'])
                                if parsed.netloc=="genius.com":
                                    try:
                                        apiurl = req_start+element['data']['api_path']
                                        annotype = urlparse(apiurl).path.split('/')[1]
                                        if annotype == 'albums':
                                            referenced['albums'].append(apiurl.encode('utf-8'))
                                        elif annotype == 'artists':
                                            referenced['artists'].append(apiurl.encode('utf-8'))
                                        elif annotype == 'songs':
                                            if urlparse(element['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                                referenced['songs'].append(apiurl.encode('utf-8'))
                                            else:
                                                referenced['other'].append(apiurl.encode('utf-8'))
                                    except:
                                        continue
                                            
    return dict(referenced)

In [7]:
def description_refs(song):
    description = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/songs/'+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for paragraph in refs_json['response']['song']['description']['dom']['children']:
        if 'children' in paragraph:
            for child in paragraph['children']:
                if 'attributes' in child:
                    if 'href' in child['attributes']:
                        url = child['attributes']['href']
                        if urlparse(url).netloc=='genius.com':
                            try:
                                apiurl = req_start+child['data']['api_path']
                                annotype = urlparse(apiurl).path.split('/')[1]
                                if annotype == 'albums':
                                    description['albums'].append(apiurl.encode('utf-8'))
                                elif annotype == 'artists':
                                    description['artists'].append(apiurl.encode('utf-8'))
                                elif annotype == 'songs':
                                    if urlparse(child['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                        description['songs'].append(apiurl.encode('utf-8'))
                                    else:
                                        description['other'].append(apiurl.encode('utf-8'))
                            except:
                                continue
    return dict(description)

In [8]:
# pass one of the two following:
# song['references'] for referents or
# song['description_refs'] for references in description

def get_neighbors(song):
    neighbors = defaultdict(list)
    for item, vals in song.iteritems():
        if item=='artists':
            for artist in vals:
                artistid = artist.rpartition('/')[2]
                if s.find({'featured_artists': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'featured_artists': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'featured_artists': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['featured_artists'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'writers': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'writers': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'writers': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['writers'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'producers': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'producers': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'producers': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['producers'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'artist_id': int(artistid)}, {'artist':1}).count():
                    name = s.find({'artist_id': int(artistid)}, {'artist': 1}).next()
                    neighbors['artist'].append({'name': name['artist'], 'id': artistid})
                else:
                    print "calling artistid: ", artistid
                    time.sleep(7)
                    name = api_call(artist)['response']['artist']['name']
                    print "successful request!"
                    print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
        elif item == 'albums':
            for album in vals:
                albumid = album.rpartition('/')[2]
                if s.find({'album.id': int(albumid)}).count():
                    name = s.find({'album.id': int(albumid)}, {'album.name': 1}).next()
                    albumname = name['album']['name']
                    neighbors['album'].append({'name': albumname, 'id': albumid})
                else:
                    print "calling albumid: ", albumid
                    time.sleep(7)
                    name = api_call(album)['response']['album']['name']
                    print "successful request!"
                    print "album name: ", name
                    neighbors['album'].append({'name': name, 'id': albumid})
        elif item == 'songs':
            for song in vals:
                songid = song.rpartition('/')[2]
                if s.find({'id': int(songid)}).count(): 
                    data = s.find({'id': int(songid)}, {'title':1, 'artist':1}).next()
                    title = data['title']
                    artist = data['artist']
                    neighbors['song'].append({'title':title, 'artist':artist, 'id':songid})
                else:
                    neighbors['song_unannotated'].append({'id':songid})
        elif item == 'other':
            for other in vals:
                otherid = other.rpartition('/')[2]
                print "calling other: ", other
                name = api_call(other)['response']['song']['title']
                neighbors['other'].append({'title':name, 'id':otherid})
                
    return neighbors

In [9]:
def annotate_song(songid):
    if s.find({'id':songid}).count() > 0:
        print "already in Mongo... returning Mongo query"
        return s.find({'id':songid}).next()
    else:
        song_info = {}
        myrequest = req_start+'/songs/'+str(songid)
        r = requests.get(myrequest, headers={'Authorization':auth})
        song_json = json.loads(r.content)
        song_info['title'] = str(song_json['response']['song']['title'])
        song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
        song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
        pickle_title = '_'.join(song_info['title'].split())+"_"+'_'.join(song_info['artist'].split())+".pkl"
        pickleDump(song_json, pickle_title)
        if song_json['response']['song']['album'] != None:
            song_info['album'] = {'name': str(song_json['response']['song']['album']['name']), 
                              'id': song_json['response']['song']['album']['id']}
        song_info['id'] = song_json['response']['song']['id']

        # may or may not exist
        song_info['featured_artists'] = []
        song_info['writers'] = []
        song_info['producers'] = []
        
        if song_json['response']['song']['sampled_songs']:
            song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
        if song_json['response']['song']['sampling_songs']:
            song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']

        for artist in song_json['response']['song']['featured_artists']:
            if artist['name']!=song_info['artist']:
                song_info['featured_artists'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['producer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['producers'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['writer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['writers'].append({'artist':artist['name'], 'id':artist['id']})

        if not song_info['featured_artists']:
            del song_info['featured_artists']
        if not song_info['writers']:
            del song_info['writers']
        if not song_info['producers']:
            del song_info['producers']

        # references!!
        time.sleep(7)
        song_info['references'] = dict(get_neighbors(get_annotations(songid)))
        time.sleep(7)
        song_info['description_refs'] = dict(get_neighbors(description_refs(song_json)))
        
        pickled_annotation = pickle_title[0:-4]+"_annotated.pkl"
        pickleDump(song_info, pickled_annotation)
        return song_info

In [36]:
def update_annotation(song):
    song_json = pickleLoad(song[0])
    song_info = pickleLoad(song[1])
    
    if song_json['response']['song']['sampled_songs']:
        song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
    if song_json['response']['song']['sampling_songs']:
        song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']
        
    song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
    song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
    
    return song_info

In [14]:
c = s.find({'description_refs.song_unannotated': {'$exists': 'true'}},
        {'description_refs.song_unannotated': 1, '_id':0})

In [15]:
r = s.find({'references.song_unannotated': {'$exists': 'true'}},
        {'references.song_unannotated': 1, '_id':0})

In [13]:
unannotated_refs = []
unannotated_desc = []

In [16]:
for i in list(c):
    for song in i['description_refs']['song_unannotated']:
        unannotated_desc.append(int(song['id']))
for i in list(r):
    for song in i['references']['song_unannotated']:
        unannotated_refs.append(int(song['id']))

In [19]:
total_unannotated = set(unannotated_desc+unannotated_refs)

In [20]:
total_unannotated = list(total_unannotated)

In [21]:
len(total_unannotated)

128

In [121]:
def insert_neighbors(neighbors):
    cant_insert = []
    for songid in neighbors:
        if s.find({'id':songid}).count()>0:
            print "song id ", songid, " in Mongo already"
            continue
        else:
            try:
                annotated = annotate_song(songid)
                s.insert_one(annotated)
                print "inserted ", songid, " to Mongo!!"
                to_insert = {'id':songid, 'title':annotated['title'], 'artist':annotated['artist']}
                s.update({'references.song_unannotated':{'$elemMatch':{'id':str(songid)}}},
                         {'$push': {'references.song': to_insert},
                          '$unset': {'references.song_unannotated': {'$elemMatch':{'id':str(songid)}}}
                         })
                s.update({'description_refs.song_unannotated':{'$elemMatch':{'id':str(songid)}}},
                 {'$push': {'description_refs.song': to_insert
                            },
                  '$unset': {'description_refs.song_unannotated': {'$elemMatch':{'id':str(songid)}}}
                 })
                time.sleep(7)
            except:
                cant_insert.append(songid)
                continue

In [363]:
refs_pickles = []
desc_pickles = []
for docid in updated_desc:
    data = s.find({'id':docid}, {'title':1, 'artist':1, '_id':0}).next()
    title = data['title']
    artist = data['artist']
    picklename = '_'.join(title.split())+"_"+'_'.join(artist.split())+"_annotated.pkl"
    desc_pickles.append(picklename)
for docid in updated_refs:
    data = s.find({'id':docid}, {'title':1, 'artist':1, '_id':0}).next()
    title = data['title']
    artist = data['artist']
    picklename = '_'.join(title.split())+"_"+'_'.join(artist.split())+"_annotated.pkl"
    refs_pickles.append(picklename)

In [382]:
def fix_ref_annotations(picklesong):
    try:
        song_info = pickleLoad(picklesong)
        ref_songs = []
        failed_songs = []
        for element in song_info['references']['song_unannotated']:
            for k, v in element.iteritems():
                if s.find({'id':int(v)}).count() > 0:
                    vals = s.find({'id':int(v)}, {'artist':1, 'title':1, '_id':0}).next()
                    to_insert = {'id':v, 'artist':vals['artist'], 'title':vals['title']}
                    ref_songs.append(to_insert)
                else:
                    failed_songs.append({'id': v})     
        songid = song_info['id']
        if ref_songs and failed_songs:
            s.update_one({'id':songid}, {'$unset': {'references.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'references.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'references.song': {'$each': ref_songs}}}) 
            s.update_one({'id':songid}, {'$addToSet': {'references.song_unannotated': {'$each': failed_songs}}})
            song_info['references']['song'] = ref_songs
            song_info['references']['song_unannotated'] = failed_songs
        elif ref_songs and not failed_songs:
            s.update_one({'id':songid}, {'$unset': {'references.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'references.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'references.song': {'$each': ref_songs}}})
            song_info['references']['song'] = ref_songs
            del song_info['references']['song_unannotated']
        elif failed_songs and not ref_songs:
            s.update_one({'id':songid}, {'$unset': {'references.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'references.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'references.song_unannotated': {'$each': failed_songs}}})
            del song_info['references']['song']
            song_info['references']['song_unannotated'] = failed_songs
        
        pickleDump(song_info, picklesong)
        return True
    except:
        print "unable to update: ", picklesong
        return False

In [354]:
def fix_desc_annotations(picklesong):
    try: 
        song_info = pickleLoad(picklesong)
        desc_songs = []
        failed_songs = []
        for element in song_info['description_refs']['song_unannotated']:
            for k, v in element.iteritems():
                if s.find({'id':int(v)}).count() > 0:
                    vals = s.find({'id':int(v)}, {'artist':1, 'title':1, '_id':0}).next()
                    to_insert = {'id':v, 'artist':vals['artist'], 'title':vals['title']}
                    desc_songs.append(to_insert)
                else:
                    failed_songs.append({'id': v})
        songid = song_info['id']
        if desc_songs and failed_songs:
            s.update_one({'id':songid}, {'$unset': {'description_refs.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'description_refs.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song': {'$each': desc_songs}}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song_unannotated': {'$each': failed_songs}}})
            song_info['description_refs']['song'] = desc_songs
            song_info['description_refs']['song_unannotated'] = failed_songs
        elif desc_songs and not failed_songs:
            s.update_one({'id':songid}, {'$unset': {'description_refs.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'description_refs.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song': {'$each': desc_songs}}})
            song_info['description_refs']['song'] = desc_songs
            del song_info['description_refs']['song_unannotated']
        elif failed_songs and not desc_songs:
            s.update_one({'id':songid}, {'$unset': {'description_refs.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'description_refs.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song_unannotated': {'$each': failed_songs}}})
            del song_info['description_refs']['song']
            song_info['description_refs']['song_unannotated'] = failed_songs
        
        pickleDump(song_info, picklesong)
        return True
    except:
        print "unable to update: ", picklesong
        return False

In [23]:
in_mongo = []
#Panda by Desiigner, Slappin by E-40, untitled by kendrick
unable = [2400811, 2421937, 639247]

In [24]:
for x in total_unannotated:
    if x in unable:
        total_unannotated.remove(x)
    if s.find({'id':x}).count()>1:
        in_mongo.append(x)
        total_unannotated.remove(x)

In [25]:
print len(in_mongo)
print len(total_unannotated)

0
127


In [26]:
#songs in mongo now
len(list(s.find()))

115

In [27]:
failed_insertion = []

In [29]:
for x in total_unannotated:
    try:
        annotated = annotate_song(x)
        s.insert_one(annotated)
        print "inserted ", annotated['title'], " by ", annotated['artist'], " with songid ", x, " to Mongo!!"
    except:
        failed_insertion.append(x)

inserted  Say My Name  by  Destiny's Child  with songid  140800  to Mongo!!
inserted  The Long And Winding Road  by  The Beatles  with songid  123451  to Mongo!!
inserted  Up All Night  by  Drake  with songid  517  to Mongo!!
inserted  Cashin Out (Freestyle)  by  Wale  with songid  68615  to Mongo!!
calling albumid:  30206
successful request!
album name:  Old 
calling artistid:  1218
successful request!
artist name:  Trick Trick
calling artistid:  1422
successful request!
artist name:  Danny Brown
calling albumid:  114275
successful request!
album name:  ShadyXV
inserted  Detroit Vs Everybody  by  Eminem  with songid  576535  to Mongo!!
inserted  Break Bread feat King Vory  by  Bryson Tiller  with songid  2149919  to Mongo!!
calling artistid:  1630
successful request!
artist name:  Ciara
calling artistid:  49470
successful request!
artist name:  DJ Esco
calling artistid:  1
successful request!
artist name:  Cam'ron
calling other:  https://api.genius.com/songs/71641
inserted  March Madn

In [31]:
success = [x for x in total_unannotated if x not in failed_insertion]

In [36]:
elem = str(success[0])

In [40]:
elem

'140800'

In [53]:
s.find({'references.song_unannotated': {'$elemMatch':{'id':elem}}}).next()

{u'_id': ObjectId('56f046e989574207d310eeb5'),
 u'album': {u'id': 6336, u'name': u'Loud'},
 u'artist': u'Rihanna',
 u'artist_id': 89,
 u'description_refs': {},
 u'featured_artists': [{u'artist': u'Drake', u'id': 130}],
 u'id': 1812,
 u'producers': [{u'artist': u'StarGate', u'id': 27578}],
 u'references': {u'other': [{u'id': u'672', u'title': u'Best I Ever Had'}],
  u'song': [{u'artist': u"Destiny's Child",
    u'id': 140800,
    u'title': u'Say My Name'}],
  u'song_unannotated': [{u'id': u'140800'}]},
 u'title': u"What's My Name?",
 u'writers': [{u'artist': u'Ester Dean', u'id': 7672},
  {u'artist': u'Traci Hale', u'id': 648546},
  {u'artist': u'Drake', u'id': 130},
  {u'artist': u'StarGate', u'id': 27578}]}

In [50]:
s.update_one({'id': 1812}, 
             {'$unset': {'references.song': ''}})

In [47]:
destiny = s.find({'id':float(elem)}).next()

In [48]:
to_insert = {'id':destiny['id'], 'artist':destiny['artist'], 'title':destiny['title']}

In [49]:
to_insert

{'artist': u"Destiny's Child", 'id': 140800, 'title': u'Say My Name'}

In [52]:
s.update_one({'id': 1812}, 
             {'$addToSet': {'references.song': to_insert}})

In [54]:
elem

'140800'

In [55]:
s.update_one({'id': 1812},
            {'$pull':{'references.song_unannotated':{'id':elem}}})

In [57]:
s.find({'$and':[{'id':1812}, {'references.song_unannotated':[]}]}).next()

{u'_id': ObjectId('56f046e989574207d310eeb5'),
 u'album': {u'id': 6336, u'name': u'Loud'},
 u'artist': u'Rihanna',
 u'artist_id': 89,
 u'description_refs': {},
 u'featured_artists': [{u'artist': u'Drake', u'id': 130}],
 u'id': 1812,
 u'producers': [{u'artist': u'StarGate', u'id': 27578}],
 u'references': {u'other': [{u'id': u'672', u'title': u'Best I Ever Had'}],
  u'song': [{u'artist': u"Destiny's Child",
    u'id': 140800,
    u'title': u'Say My Name'}],
  u'song_unannotated': []},
 u'title': u"What's My Name?",
 u'writers': [{u'artist': u'Ester Dean', u'id': 7672},
  {u'artist': u'Traci Hale', u'id': 648546},
  {u'artist': u'Drake', u'id': 130},
  {u'artist': u'StarGate', u'id': 27578}]}

In [59]:
s.update_one({'$and':[{'id':1812}, {'references.song_unannotated':[]}]},
            {'$unset': {'references.song_unannotated':''}})

In [61]:
s.find({'id':1812}).next()

{u'_id': ObjectId('56f046e989574207d310eeb5'),
 u'album': {u'id': 6336, u'name': u'Loud'},
 u'artist': u'Rihanna',
 u'artist_id': 89,
 u'description_refs': {},
 u'featured_artists': [{u'artist': u'Drake', u'id': 130}],
 u'id': 1812,
 u'producers': [{u'artist': u'StarGate', u'id': 27578}],
 u'references': {u'other': [{u'id': u'672', u'title': u'Best I Ever Had'}],
  u'song': [{u'artist': u"Destiny's Child",
    u'id': 140800,
    u'title': u'Say My Name'}]},
 u'title': u"What's My Name?",
 u'writers': [{u'artist': u'Ester Dean', u'id': 7672},
  {u'artist': u'Traci Hale', u'id': 648546},
  {u'artist': u'Drake', u'id': 130},
  {u'artist': u'StarGate', u'id': 27578}]}

In [66]:
success[0]

140800

In [67]:
def update_annotations(songs):
    for songid in songs:
        try:
            thesong = s.find({'id':songid}).next()
            songstr = str(songid)
            to_insert = {'id':songid, 'artist':thesong['artist'], 'title':thesong['title']}
            s.update_many({'references.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
             {'$addToSet': {'references.song': to_insert}})
            s.update_many({'description_refs.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
             {'$addToSet': {'description_refs.song': to_insert}})
            s.update_many({'references.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
             {'$pull': {'references.song_unannotated': songid}})
            s.update_many({'description_refs.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
             {'$pull': {'description_refs.song_unannotated': songid}})
            s.update_many({'references.song_unannotated':[]},
                         {'$unset': {'references.song_unannotated':''}})
            s.update_many({'description_refs.song_unannotated':[]},
                         {'$unset': {'description_refs.song_unannotated':''}})
        except:
            print "unable to update: ", songid
            continue

In [69]:
songid = success[0]
try:
    thesong = s.find({'id':songid}).next()
    songstr = str(songid)
    to_insert = {'id':songid, 'artist':thesong['artist'], 'title':thesong['title']}
    s.update_many({'references.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
     {'$addToSet': {'references.song': to_insert}})
    s.update_many({'description_refs.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
     {'$addToSet': {'description_refs.song': to_insert}})
    s.update_many({'references.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
     {'$pull': {'references.song_unannotated': songid}})
    s.update_many({'description_refs.song_unannotated': {'$elemMatch':{'id':songstr}}}, 
     {'$pull': {'description_refs.song_unannotated': songid}})
    s.update_many({'references.song_unannotated':[]},
                 {'$unset': {'references.song_unannotated':''}})
    s.update_many({'description_refs.song_unannotated':[]},
                 {'$unset': {'description_refs.song_unannotated':''}})
except:
    print "unable to update: ", songid

In [70]:
update_annotations(success)

In [72]:
sampled_songs = list(s.find({'sampling_songs':{ '$exists': 'true', '$ne': [] }}, \
                            {'sampling_songs':1, '_id':0}))

In [82]:
for samp in sampled_songs:
    for w in samp:
        for song in samp[w]:
            sampling_songs.append(song['id'])

In [83]:
print "before removing existing: ", len(sampling_songs)
for songid in sampling_songs:
    if s.find({'id':songid}).count() > 0:
        sampling_songs.remove(songid)
print "after removing existing: ", len(sampling_songs)

before removing existing:  20
after removing existing:  17


In [84]:
print len(sampled_songs)

11
